In [2]:
## reading the images files of train folder

import os
import cv2


path = r"G:\Signature Verification Project\data\gen_forge\test"  ## data/gen_forge
unique_id = [] ## name or id of each image (same as the id in train_data.csv)
images = [] ## image array

for folder, a, b in os.walk(path):
    for file in b:
        img = cv2.imread(os.path.join(folder, file)) ## reading image as matrix
        
        if img is not None:
            images.append(img)
            
            ## creating  unique id for each image
            if "forg" in str(folder):
                unique_id.append(str(folder[-8:]) + "/" + str(file.lower()))
            else:
                unique_id.append(str(folder[-3:]) + "/" + str(file.lower()))


## CLAHE image enhancement
clahe = cv2.createCLAHE(clipLimit =2.0, tileGridSize=(8,8))


## grayscaling and resizing the images

resized_images = []
dimension = (480, 360) ## new size (width, height)

for img in images:
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) ## grayscaling
    
    resized_image = cv2.resize(img, dimension, interpolation = cv2.INTER_AREA) ## resizing
    resized_image = clahe.apply(resized_image)
    
    resized_images.append(resized_image)
    

print(resized_images[0].shape)  ## (height, width)

## finding co-ordinates

import numpy as np


height = [] ## Y coordinates in our case 
width = [] ## X coordinates in our case

for image in resized_images:   
    indices = np.where(image < 130) ## finds where signature curves lie 
    
    height.append(indices[0])
    width.append(indices[1])


## creating dataframe from above info

import pandas as pd

features_1_df = pd.DataFrame(list(zip(unique_id, width, height)), columns = ["ID", "X", "Y"])

del height, width

## exploding X & Y columns of the dataframe

features = features_1_df.set_index('ID').apply(pd.Series.explode).reset_index()


del features_1_df

## finding the position of pen-tip

import math

Xs = features["X"].tolist()
Ys = features["Y"].tolist()

Rs = []

for i in range(len(Xs)):
    Rs.append(math.sqrt(Xs[i]**2 + Ys[i]**2))

features["R"] = Rs ## "R" is the position column


## angle of "R"

theta = []

for i in range(len(Xs)):
    try:
        theta.append(np.arctan(Ys[i]/Xs[i]))
    except ZeroDivisionError:
        theta.append(0)

features["Angle"] = theta

del Xs, Ys, Rs, theta


## finding change in X direction
features['del_x'] = np.where(features.ID == features.ID.shift(1), features.X.diff(), 0)
features['del_x'] = features.groupby("ID")["del_x"].shift(-1)


## finding change in Y direction
features['del_y'] = np.where(features.ID == features.ID.shift(1), features.Y.diff(), 0)
features['del_y'] = features.groupby("ID")["del_y"].shift(-1)

## Magnitude and angle of displacement vector

del_x_l = features.del_x.tolist()
del_y_l = features.del_y.tolist()

disp = [] ## magnitude
disp_theta = [] ## angle

for i in range(len(del_x_l)):
    
    ## magnitude
    disp.append(math.sqrt(del_x_l[i]**2 + del_y_l[i]**2))
    
    ## angle
    try:
        disp_theta.append(np.arctan(del_y_l[i]/del_x_l[i]))
    except ZeroDivisionError:
        disp_theta.append(0)
    
features["displacement"] = disp
features["disp_angle"] = disp_theta


del disp_theta


cos_angle = []
sine_angle = []

for i in range(len(del_x_l)):
    
    ## cosine angle (angle between x-axis and signature curve)
    cos_angle.append((del_x_l[i])/(disp[i]))
    
    ## sine angle (angle between y-axis and signature curve)
    sine_angle.append((del_y_l[i])/(disp[i]))
    
features["cos_angle"] = cos_angle
features["sine_angle"] = sine_angle

del cos_angle, sine_angle, del_x_l, del_y_l, disp


## finding time
features["Time"] = features.groupby(["ID"]).cumcount()+1


## finding change in Time
features['del_t'] = np.where(features.ID == features.ID.shift(1), features.Time.diff(), 0)
features['del_t'] = features.groupby("ID")["del_t"].shift(-1)

del_x_l = features.del_x.tolist()
del_y_l = features.del_y.tolist()
del_t_l = features.del_t.tolist()

vel_x = [] ## velocity in x-direction
vel_y = [] ## velocity in y-direction

for i in range(len(del_x_l)):
    vel_x.append(del_x_l[i]/del_t_l[i])
    vel_y.append(del_y_l[i]/del_t_l[i])
    
features["vel_x"] = vel_x
features["vel_y"] = vel_y


del del_x_l, del_y_l, del_t_l


res_vel = []  ## resultant velocity 
dir_res_vel = []  ## direction of the resultant velocity

for i in range(len(vel_x)):
    res_vel.append(math.sqrt(vel_x[i]**2 + vel_y[i]**2))
    
    try:
        dir_res_vel.append(np.arctan(vel_y[i]/vel_x[i]))
    except ZeroDivisionError:
        dir_res_vel.append(0)
        
features["res_vel"] = res_vel
features["dir_res_vel"] = dir_res_vel


del vel_x, vel_y, res_vel, dir_res_vel


del_t_l = features.del_t.tolist()

## finding change in displacement angle
features['del_disp_angle'] = np.where(features.ID == features.ID.shift(1), features.disp_angle.diff(), 0)
features['del_disp_angle'] = features.groupby("ID")["del_disp_angle"].shift(-1)
## converting it to list
del_disp_angle = features.del_disp_angle.tolist()

angular_vel = [] ## angular velocity

for i in range(len(del_disp_angle)):
    angular_vel.append(del_disp_angle[i]/del_t_l[i])

features["angular_vel"] = angular_vel


del del_disp_angle, del_t_l, angular_vel


del_vel_x = features.vel_x.tolist()
del_vel_y = features.vel_y.tolist()
del_t_l = features.del_t.tolist()

acceleration_x = [] ## acceleration in x-dir
acceleration_y = [] ## acceleration in y-dir
res_acceleration = [] ## resultant acceleration

for i in range(len(del_t_l)):
    acceleration_x.append(del_vel_x[i]/del_t_l[i])
    acceleration_y.append(del_vel_y[i]/del_t_l[i])
    
    res_acceleration.append(math.sqrt(acceleration_x[i]**2 + acceleration_y[i]**2))
    
features["acceleration_x"] = acceleration_x
features["acceleration_y"] = acceleration_y
features["res_acceleration"] = res_acceleration

del del_vel_x, del_vel_y, del_t_l, acceleration_x, acceleration_y, res_acceleration


angular_vel = features.angular_vel.tolist()
displacement = features.displacement.tolist()
res_acceleration = features.res_acceleration.tolist()

centripetal_accln = [] ## centripetal acceleration
tangential_accln = [] ## tangential acceleration

for i in range(len(angular_vel)):
    centripetal_accln.append((angular_vel[i]**2)*displacement[i])
    
    tangential_accln.append(math.sqrt(abs(res_acceleration[i]**2 - centripetal_accln[i]**2)))
    
features["centripetal_accln"] = centripetal_accln
features["tangential_accln"] = tangential_accln

del angular_vel, res_acceleration, displacement, centripetal_accln, tangential_accln


features = features.drop(["Time", "del_t", "del_disp_angle"], axis = 1)


features["Angle"] = features["Angle"].fillna(0)
features["disp_angle"] = features["disp_angle"].fillna(0)
features["dir_res_vel"] = features["dir_res_vel"].fillna(0)


print(features.head())

features.to_csv(r"G:\Signature Verification Project\data\Final Data\CLAHE\Testing\gen_forg_test.csv", index = False) ## gen_forg data


print("complete")


(360, 480)


c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:130: RuntimeWarning: divide by zero encountered in longlong_scalars
c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:191: RuntimeWarning: divide by zero encountered in double_scalars


                    ID    X  Y           R     Angle del_x del_y  \
0  test\049/01_049.png  208  2  208.009615  0.009615    -1     1   
1  test\049/01_049.png  207  3  207.021738  0.014492     1     0   
2  test\049/01_049.png  208  3  208.021633  0.014422    -3     1   
3  test\049/01_049.png  205  4  205.039021  0.019510     1     0   
4  test\049/01_049.png  206  4  206.038831  0.019415     1     0   

   displacement  disp_angle  cos_angle  ...  vel_x  vel_y   res_vel  \
0      1.414214   -0.785398  -0.707107  ...   -1.0    1.0  1.414214   
1      1.000000    0.000000   1.000000  ...    1.0    0.0  1.000000   
2      3.162278   -0.321751  -0.948683  ...   -3.0    1.0  3.162278   
3      1.000000    0.000000   1.000000  ...    1.0    0.0  1.000000   
4      1.000000    0.000000   1.000000  ...    1.0    0.0  1.000000   

   dir_res_vel  angular_vel  acceleration_x  acceleration_y  res_acceleration  \
0    -0.785398     0.785398            -1.0             1.0          1.414214   
1 